**1. CNN for Image Classification**

In [1]:
# Step 1: Set up environment

import torch
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
import numpy as np

In [ ]:
# Step 2: Prepare dataset
# Define data transformations with feature scaling
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the CIFAR-10 dataset
Train_dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)
Test_dataset = CIFAR10(root='./data', train=False, transform=transform, download=True)

In [ ]:
# Step 3: Split dataset
# define sizes of train, validation, test datasets
train_size = int(0.6 * len(Train_dataset))
val_size = int(0.2 * len(Train_dataset))
test_size = int(0.2 * len(Train_dataset))


# Splitting datasets
train_dataset, val_dataset, test_dataset = random_split(Train_dataset, [train_size, val_size, test_size])

# Convert datasets to DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [ ]:
# Step 4: Building CNN
class CNN(nn.Module):
    def __init__(self, x1, m1, x2, m2, x3, x4, x5, d):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, x1, kernel_size=m1, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(x1, x2, kernel_size=m2, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(x2 * 8 * 8, x3)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=d)

        self.fc2 = nn.Linear(x3, x4)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(p=d)

        self.fc3 = nn.Linear(x4, x5)
        self.relu = nn.ReLU()
        self.dropout3 = nn.Dropout(p=d)

        self.fc4 = nn.Linear(x5, 10)  # 10 classes for CIFAR-10
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, x2 * 8 * 8)

        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        x = F.relu(self.fc2(x))
        x = self.dropout2(x)

        x = F.relu(self.fc3(x))
        x = self.dropout3(x)

        x = self.fc4(x)
        return self.softmax(x)

In [ ]:
# Step 5: Determine the parameters
x1, m1, x2, m2, x3, x4, x5, d = 32, 3, 64, 3, 4096, 512, 128, 0.4

In [ ]:
model = CNN(x1, m1, x2, m2, x3, x4, x5, d)
print(model)

In [ ]:
# Step 6: Train the model
def train_model(model, train_loader, val_loader, epochs, lr):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_losses, val_losses = [], []

    for epoch in range(epochs):
        # Training Step
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # Backward and Optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_losses.append(running_loss / len(train_loader))

        # Evaluation Step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_losses.append(val_loss / len(val_loader))

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_losses[-1]}, Val Loss: {val_losses[-1]}')

    return train_losses, val_losses

In [ ]:
# Train the model
train_losses, val_losses = train_model(model, train_loader, val_loader, epochs=20, lr=0.001)

# Plot training and validation loss
plt.plot(range(1, 21), train_losses, label='Training Loss')
plt.plot(range(1, 21), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Step 7: Evaluate the Model
model.eval()
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')

print(f'Test Accuracy: {accuracy*100} %')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

In [ ]:
# Step 8: Plot training and validation loss for different learning rates
learning_rates = [0.0001, 0.001, 0.01, 0.1]

fig = plt.figure(figsize=(20, 4))
i = 0

for lr in learning_rates:
    print(f"For Learning Rate = {lr}")
    model = CNN(x1, m1, x2, m2, x3, x4, x5, d)
    train_losses, val_losses = train_model(model, train_loader, val_loader, epochs=8, lr=lr)

    i += 1
    fig.add_subplot(1,4,i)
    plt.plot(range(1, 9), train_losses, label=f'Training Loss, LR={lr}')
    plt.plot(range(1, 9), val_losses, label=f'Validation Loss, LR={lr}')
    plt.title(f"Learning Rate = {lr}")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

plt.show()

